# The Intelligence Bound: Interactive Reproduction

This notebook reproduces all numerical results from:

> **The Intelligence Bound: Thermodynamic Limits on Learning Rate and Implications for Biosphere Information**
>
> Justin Hart, Viridis LLC

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from code.intelligence_bound import (
    landauer_bound,
    data_bound,
    intelligence_bound,
    system_analysis,
    temperature_scaling_table,
    real_systems_table,
    critical_power,
    BOLTZMANN_CONSTANT,
    LN2
)

from code.simulations import (
    biodiversity_impact,
    strategy_comparison,
    biosphere_information_potential,
    crossover_time,
    cascade_model
)

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## The Intelligence Bound

The main theorem establishes:

$$\dot{I} \leq \min\left(D \cdot B, \frac{P}{k_B T \ln 2}\right)$$

where:
- $D$ = data richness (predictive fraction) ∈ [0, 1]
- $B$ = observation bandwidth [bits/s]
- $P$ = power [W]
- $T$ = temperature [K]

## Table I: Temperature Scaling

In [ ]:
table = temperature_scaling_table(power=1.0)

df = pd.DataFrame({
    'Temperature [K]': table['temperature_K'],
    'İ_max [bits/s]': [f'{x:.2e}' for x in table['I_max_bits_per_s']],
    'İ_max × T': [f'{x:.2e}' for x in table['I_max_times_T']]
})

print("Table I: Temperature Scaling Validation (P = 1 W)")
print("="*50)
display(df)

print(f"\nTheoretical prediction: İ×T = P/(k_B ln 2) = {1/(BOLTZMANN_CONSTANT * LN2):.2e}")
print("✓ Product is constant across temperatures")

In [ ]:
# Visualize temperature scaling
temperatures = np.logspace(0, 4, 100)  # 1 K to 10000 K
I_max = [landauer_bound(1.0, T) for T in temperatures]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# İ_max vs T
axes[0].loglog(temperatures, I_max, 'b-', linewidth=2)
axes[0].set_xlabel('Temperature [K]')
axes[0].set_ylabel('İ_max [bits/s]')
axes[0].set_title('Landauer Bound vs Temperature')
axes[0].axvline(300, color='r', linestyle='--', label='Room temp')
axes[0].legend()

# İ×T = constant
product = [I * T for I, T in zip(I_max, temperatures)]
axes[1].semilogx(temperatures, product, 'g-', linewidth=2)
axes[1].set_xlabel('Temperature [K]')
axes[1].set_ylabel('İ_max × T')
axes[1].set_title('Constant Product (Landauer Prediction)')
axes[1].axhline(product[0], color='r', linestyle='--', label='Theoretical')
axes[1].legend()

plt.tight_layout()
plt.show()

## Table II: Real System Analysis

In [ ]:
systems = real_systems_table()

data = []
for name, s in systems.items():
    data.append({
        'System': name,
        'Data Bound': f'{s.data_bound:.2e}',
        'Landauer Bound': f'{s.landauer_bound:.2e}',
        'Headroom': f'{s.headroom:.2e}',
        'Limiting': s.limiting_factor
    })

df = pd.DataFrame(data)
print("Table II: Real System Analysis")
print("="*70)
display(df)

print("\n✓ All systems are DATA-limited, not POWER-limited")
print(f"✓ Headroom ranges from 10^12 to 10^16")

In [ ]:
# Visualize headroom
names = list(systems.keys())
headrooms = [systems[n].headroom for n in names]

fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.barh(names, [np.log10(h) for h in headrooms], color=['#2ecc71', '#3498db', '#9b59b6'])
ax.set_xlabel('log₁₀(Headroom)')
ax.set_title('Distance from Thermodynamic Limit')

for i, (bar, h) in enumerate(zip(bars, headrooms)):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'{h:.0e}×', va='center')

ax.set_xlim(0, 20)
plt.tight_layout()
plt.show()

## Table IV: Biodiversity Impact

In [ ]:
results = biodiversity_impact(D0=0.5)

data = []
for r in results:
    data.append({
        'Biodiversity Loss': f'{r.loss_fraction:.0%}',
        'D_after': f'{r.D_after:.3f}',
        'Ceiling After': f'{r.ceiling_after:.2e}',
        'Reduction': f'{r.ceiling_reduction_pct:.1f}%'
    })

df = pd.DataFrame(data)
print("Table IV: Biodiversity Loss Impact on Intelligence Ceiling")
print("="*60)
display(df)

In [ ]:
# Visualize cascade effect
loss_fractions = np.linspace(0, 0.99, 100)
D0 = 0.5

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Different cascade strengths
for beta in [0, 1, 2, 4]:
    D_values = [cascade_model(D0, f, beta) for f in loss_fractions]
    label = 'Linear' if beta == 0 else f'Cascade (β={beta})'
    axes[0].plot(loss_fractions * 100, D_values, label=label, linewidth=2)

axes[0].set_xlabel('Biodiversity Loss [%]')
axes[0].set_ylabel('Data Richness D')
axes[0].set_title('Nonlinear Cascade Effect')
axes[0].legend()
axes[0].set_xlim(0, 100)
axes[0].set_ylim(0, D0 * 1.1)

# Ceiling reduction
D_cascade = [cascade_model(D0, f, 2) for f in loss_fractions]
ceiling_reduction = [(1 - d/D0) * 100 for d in D_cascade]

axes[1].plot(loss_fractions * 100, ceiling_reduction, 'r-', linewidth=2)
axes[1].plot(loss_fractions * 100, loss_fractions * 100, 'k--', label='Linear (no cascade)')
axes[1].fill_between(loss_fractions * 100, loss_fractions * 100, ceiling_reduction,
                     alpha=0.3, color='red', label='Cascade amplification')
axes[1].set_xlabel('Biodiversity Loss [%]')
axes[1].set_ylabel('Intelligence Ceiling Reduction [%]')
axes[1].set_title('Ceiling Reduction vs Biodiversity Loss')
axes[1].legend()

plt.tight_layout()
plt.show()

## Table V: Strategy Comparison

In [ ]:
strategies = strategy_comparison(D0=0.5, time_horizon=200)

data = []
for name, r in strategies.items():
    data.append({
        'Strategy': name,
        'Final D': f'{r.final_D:.3f}',
        'Total Intelligence': f'{r.total_intelligence:.2e}',
        'Outcome': r.outcome
    })

df = pd.DataFrame(data)
print("Table V: Strategy Comparison over 200 Years")
print("="*60)
display(df)

I_exploit = strategies['Exploit'].total_intelligence
I_sustain = strategies['Sustain'].total_intelligence
I_regen = strategies['Regenerate'].total_intelligence

print(f"\nRatio Sustain/Exploit: {I_sustain/I_exploit:.1f}×")
print(f"Ratio Regenerate/Exploit: {I_regen/I_exploit:.1f}×")

In [ ]:
# Visualize strategies
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {'Exploit': '#e74c3c', 'Sustain': '#3498db', 'Regenerate': '#2ecc71'}

# D trajectories
for name, r in strategies.items():
    axes[0].plot(r.time_points, r.D_trajectory, color=colors[name], 
                 label=name, linewidth=2)

axes[0].set_xlabel('Time [years]')
axes[0].set_ylabel('Data Richness D')
axes[0].set_title('Biosphere D Trajectories')
axes[0].legend()
axes[0].set_ylim(0, 1.1)

# Cumulative intelligence
SECONDS_PER_YEAR = 365.25 * 24 * 3600
bandwidth = 1e12

for name, r in strategies.items():
    cumulative = np.zeros_like(r.time_points)
    dt = r.time_points[1] - r.time_points[0]
    for i in range(1, len(r.time_points)):
        cumulative[i] = cumulative[i-1] + r.D_trajectory[i] * bandwidth * dt * SECONDS_PER_YEAR
    axes[1].plot(r.time_points, cumulative, color=colors[name], 
                 label=name, linewidth=2)

axes[1].set_xlabel('Time [years]')
axes[1].set_ylabel('Cumulative Intelligence [bits]')
axes[1].set_title('Total Intelligence Acquired')
axes[1].legend()
axes[1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

plt.tight_layout()
plt.show()

## Critical Power & Phase Transition

In [ ]:
# Calculate critical power for H100-like system
D = 0.05
B = 3e12  # bits/s
T = 350   # K

P_star = critical_power(D, B, T)
P_h100 = 700  # W

print("Phase Transition Analysis")
print("="*50)
print(f"System parameters: D = {D}, B = {B:.0e} bits/s, T = {T} K")
print(f"\nCritical power P* = D × B × k_B × T × ln(2)")
print(f"P* = {P_star:.2e} W = {P_star/1e9:.2f} GW")
print(f"\nH100 actual power: {P_h100} W")
print(f"Ratio P*/P_H100 = {P_star/P_h100:.2e}")
print(f"\n→ H100 operates {P_star/P_h100:.0e}× below critical power")

In [ ]:
# Visualize phase diagram
powers = np.logspace(-3, 15, 100)  # 1 mW to 1 PW

I_landauer = [landauer_bound(P, T) for P in powers]
I_data = data_bound(D, B)

fig, ax = plt.subplots(figsize=(10, 6))

ax.loglog(powers, I_landauer, 'b-', label='Landauer bound', linewidth=2)
ax.axhline(I_data, color='r', linestyle='-', label='Data bound', linewidth=2)
ax.axvline(P_star, color='g', linestyle='--', label=f'P* = {P_star:.2e} W', linewidth=1)
ax.axvline(P_h100, color='orange', linestyle=':', label=f'H100 ({P_h100} W)', linewidth=2)

# Shade regions
ax.fill_between(powers[powers < P_star], 0, I_data, alpha=0.2, color='red',
                label='Data-limited regime')
ax.fill_between(powers[powers >= P_star], 0, 
                [landauer_bound(P, T) for P in powers[powers >= P_star]],
                alpha=0.2, color='blue', label='Power-limited regime')

ax.set_xlabel('Power [W]')
ax.set_ylabel('Intelligence Bound [bits/s]')
ax.set_title('Phase Diagram: Data-Limited vs Power-Limited Regimes')
ax.legend(loc='lower right')
ax.set_xlim(1e-3, 1e15)
ax.set_ylim(1e8, 1e30)

plt.tight_layout()
plt.show()

## Crossover Time Analysis

In [ ]:
# Calculate crossover times for different decay rates
decay_rates = [0.01, 0.02, 0.05, 0.1]

print("Crossover Time Analysis")
print("="*50)
print("Time at which Sustain strategy beats Exploit:")
print(f"\n{'Decay Rate λ':<15} {'t* (years)':<15} {'1/λ':<12}")
print("-"*42)

for lam in decay_rates:
    t_star = crossover_time(D0=0.5, decay_rate=lam)
    print(f"{lam:<15.2f} {t_star:<15.1f} {1/lam:<12.1f}")

## Summary

### Key Findings Reproduced:

1. **Temperature Scaling (Table I)**: İ × T = constant ✓
2. **All Systems Data-Limited (Table II)**: Headroom 10¹²-10¹⁶ ✓
3. **Nonlinear Cascade (Table IV)**: 90% loss → >40% ceiling reduction ✓
4. **Strategy Ordering (Table V)**: Regenerate > Sustain > Exploit ✓
5. **Phase Transition**: P* >> current system power ✓